#### **Importing libaries**

In [10]:
import tensorflow as tf


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout ,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Reading the files**

In [16]:
# get the data
filname = './new/data/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('./new/data/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


**Preprocessing**

In [14]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y


In [17]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

7


In [8]:
# keras with tensorflow backend
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

**Train-test split**

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

**Defining our CNN model**

In [10]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()
    
    return model
model=my_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)       409728    
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512       
__________

In [11]:
path_model='model_filter.h5' # save model at this location after each epoch
K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate
# fit the model
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

Train on 32298 samples, validate on 3589 samples
Epoch 1/20
32298/32298 [==============================] - 19s 602us/step - loss: 1.6501 - acc: 0.3562 - val_loss: 1.4523 - val_acc: 0.4472
Epoch 2/20
32298/32298 [==============================] - 14s 439us/step - loss: 1.2880 - acc: 0.5137 - val_loss: 1.3007 - val_acc: 0.4990
Epoch 3/20
32298/32298 [==============================] - 14s 446us/step - loss: 1.1204 - acc: 0.5766 - val_loss: 1.1900 - val_acc: 0.5553
Epoch 4/20
32298/32298 [==============================] - 14s 436us/step - loss: 1.0000 - acc: 0.6268 - val_loss: 1.1796 - val_acc: 0.5545
Epoch 5/20
32298/32298 [==============================] - 14s 442us/step - loss: 0.8830 - acc: 0.6709 - val_loss: 1.1649 - val_acc: 0.5709
Epoch 6/20
32298/32298 [==============================] - 14s 440us/step - loss: 0.7410 - acc: 0.7312 - val_loss: 1.2765 - val_acc: 0.5483
Epoch 7/20
32298/32298 [==============================] - 14s 437us/step - loss: 0.5795 - acc: 0.7900 - val_loss: 1.1

In [12]:
model.save('modelcnn.h5')

**Feature Ectraction from CNN**

In [13]:
from keras import models
modelfeatured = models.Model(
    inputs=model.input,
    outputs=model.get_layer('max_pooling2d_3').output
)

In [14]:
train_featured = modelfeatured.predict(X_train)
test_featured = modelfeatured.predict(X_test)

In [15]:
train_featured.shape

(32298, 6, 6, 256)

In [16]:
x_train = train_featured.reshape(train_featured.shape[0], train_featured.shape[1], train_featured.shape[2]*train_featured.shape[3])
x_test = test_featured.reshape(test_featured.shape[0], test_featured.shape[1], test_featured.shape[2]*test_featured.shape[3])

In [17]:
x_train.shape

(32298, 6, 1536)

**LSTM model**

In [18]:
time_step = 6
features = 1536 
model1 = Sequential()
#model.add(Embedding(max_features, output_dim = embed_outDim, input_length = input_length))
model1.add(LSTM(256, input_shape=(time_step, features), return_sequences=True))
model.add(Dropout(0.5))

model1.add(LSTM(128, input_shape=(time_step, 128), return_sequences=True))
model1.add(Dropout(0.5))



model1.add(Flatten())


model1.add(Dense(7, activation='softmax'))


In [19]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 6, 256)            1836032   
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 128)            197120    
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 5383      
Total params: 2,038,535
Trainable params: 2,038,535
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras import optimizers
adam = optimizers.Adam(lr=0.0001, decay=1e-6)
    #Kb.set_value(model.optimizer.lr, 0.01)
    
        
        
model1.compile(loss='categorical_crossentropy',
              optimizer=adam,   #'adam',
              metrics=['accuracy'])

In [21]:
h=model1.fit(x=x_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(x_test,y_test),
            shuffle=True,
            )

Train on 32298 samples, validate on 3589 samples
Epoch 1/20
32298/32298 [==============================] - 15s 453us/step - loss: 1.2676 - acc: 0.5234 - val_loss: 1.1677 - val_acc: 0.5678
Epoch 2/20
32298/32298 [==============================] - 13s 391us/step - loss: 0.7591 - acc: 0.7268 - val_loss: 1.2042 - val_acc: 0.5890
Epoch 3/20
32298/32298 [==============================] - 13s 398us/step - loss: 0.4898 - acc: 0.8310 - val_loss: 1.3474 - val_acc: 0.5832
Epoch 4/20
32298/32298 [==============================] - 13s 391us/step - loss: 0.2822 - acc: 0.9100 - val_loss: 1.5312 - val_acc: 0.5963
Epoch 5/20
32298/32298 [==============================] - 13s 410us/step - loss: 0.1377 - acc: 0.9633 - val_loss: 1.7653 - val_acc: 0.5876
Epoch 6/20
32298/32298 [==============================] - 13s 388us/step - loss: 0.0683 - acc: 0.9857 - val_loss: 1.9227 - val_acc: 0.5938
Epoch 7/20
32298/32298 [==============================] - 13s 388us/step - loss: 0.0429 - acc: 0.9927 - val_loss: 2.1

# Testing`our model in  real time through Webcam

In [28]:
#model1.save('model_lstm.h5')
import cv2
import time

In [29]:
model=tf.keras.models.load_model('model.h5')

In [30]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Sad", 5: "Surprised", 6:"Neutral" }

In [31]:
facecasc =  cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')  

In [33]:
cap = cv2.VideoCapture(0)
time.sleep(5)
c=0
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    # Convert to grayscale  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
     # Detect the faces
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
      
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        
        roi_gray = gray[y:y + h, x:x + w]
        
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        
        
        
        prediction = model.predict(cropped_img)
        
        maxindex = int(np.argmax(prediction))
        
                
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        x=cropped_img[0].reshape(48,48)
        
        save_image(maxindex,x,c)
        
        c+=1
    cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
def save_image(emotion_no,img_array,img_no):
    
    img = Image.fromarray(img_array)


    emotion_name = emotion_dict[emotion_no]


    if not os.path.exists(f'{os.getcwd()}/emotion/{emotion_name}'):
        os.mkdir(f'{os.getcwd()}/emotion/{emotion_name}')

    img.save(f'{os.getcwd()}/emotion/{emotion_name}/{img_no}'+'.png')